<a href="https://colab.research.google.com/github/meti-94/BERT-QA/blob/main/parsbert_digikala_political.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install transformers

     |████████████████████████████████| 1.5MB 5.9MB/s 
     |████████████████████████████████| 2.9MB 44.5MB/s 
     |████████████████████████████████| 890kB 39.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=fbb4fdbe47c373d848906f5194f6dbc44ed4393c8251ec4198d445466eff2c3b
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
import pandas as pd 
from sklearn.model_selection import train_test_split
df = pd.read_excel('drive/My Drive/train.xlsx').fillna('')
# print(df.head()) 
X = df['text'].to_list(); y = df['label'].to_list()
df = pd.read_excel('drive/My Drive/taghche.xlsx').fillna('')
# print(df[['comment', 'label_id']].head()) 
X2 = df['comment'].to_list(); y2 = df['label_id'].to_list()
df = pd.read_excel('drive/MyDrive/snappfood.xlsx')
# print(df[['comment', 'label_id']].head()) 
X3 = df['comment'].to_list(); y3 = df['label_id'].apply(lambda x:1-x).to_list()
X+=X2; y+=y2
X+=X3; y+=y3
train_texts, valid_texts, train_labels, valid_labels = train_test_split(X, 
                                                                      y, 
                                                                      random_state=42, 
                                                                      stratify=y, 
                                                                      test_size=.1)
print(len(train_labels), sum(y), len(valid_labels))

127033 71174 14115


In [3]:
from transformers import AutoConfig, AutoTokenizer, AutoModel, BertForSequenceClassification

config = AutoConfig.from_pretrained("HooshvareLab/bert-base-parsbert-uncased")
tokenizer = AutoTokenizer.from_pretrained("HooshvareLab/bert-base-parsbert-uncased")
model = BertForSequenceClassification.from_pretrained("HooshvareLab/bert-base-parsbert-uncased")
# config = AutoConfig.from_pretrained("/content/drive/MyDrive/Colab Notebooks/LSCP_tiny/config.json", local_files_only=True)
# tokenizer = AutoTokenizer.from_pretrained("HooshvareLab/bert-fa-base-uncased")
# model = BertForSequenceClassification.from_pretrained('/content/drive/MyDrive/Colab Notebooks/LSCP_tiny/')

Some weights of the model checkpoint at HooshvareLab/bert-base-parsbert-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassifica

In [4]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=50)
val_encodings = tokenizer(valid_texts, truncation=True, padding=True, max_length=50)

In [5]:
import torch

class IMDbDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = IMDbDataset(train_encodings, train_labels)
valid_dataset = IMDbDataset(val_encodings, valid_labels)

In [6]:
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=5,              # total number of training epochs
    per_device_train_batch_size=150,  # batch size per device during training
    per_device_eval_batch_size=150,   # batch size for evaluation
    # warmup_steps=500,                # number of warmup steps for learning rate scheduler
    # weight_decay=0.01,               # strength of weight decay
    learning_rate= 5e-6,
    adam_epsilon = 1e-8, 
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
    do_eval=True,
    evaluation_strategy = 'epoch'
    
    
)
# training_args.evaluation_strategy = EvaluationStrategy.EPOCH
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=valid_dataset,             # evaluation dataset
#     evaluation_strategy='epoch',
)

trainer.train()

Epoch,Training Loss,Validation Loss
1,0.318532,0.324783
2,0.284985,0.317027
3,0.298803,0.310597
4,0.275990,0.311076
5,0.255019,0.312908


TrainOutput(global_step=4235, training_loss=0.29890197778677013)

In [7]:
df = pd.read_excel('drive/My Drive/test.xlsx').fillna('')
X = df['text'].to_list(); y = df['label'].to_list()

In [8]:
test_encodings = tokenizer(X, truncation=True, padding=True, max_length=128)

In [9]:
test_dataset = IMDbDataset(test_encodings, y)

In [10]:
pred = trainer.predict(test_dataset)

In [11]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [12]:
compute_metrics(pred)

{'accuracy': 0.8172823218997362,
 'f1': 0.6506935687263556,
 'precision': 0.5454545454545454,
 'recall': 0.80625}

In [13]:
from sklearn.metrics import classification_report
print(classification_report(pred.label_ids, pred.predictions.argmax(-1)))

              precision    recall  f1-score   support

           0       0.94      0.82      0.88      1196
           1       0.55      0.81      0.65       320

    accuracy                           0.82      1516
   macro avg       0.74      0.81      0.76      1516
weighted avg       0.86      0.82      0.83      1516



In [14]:
trainer.save_model("./finetuned_parsbert")

In [15]:
cp -r finetuned_parsbert/ drive/MyDrive/